# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 5079, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 5079 (delta 74), reused 82 (delta 32), pack-reused 4928 (from 3)
Receiving objects: 100% (5079/5079), 379.22 MiB | 39.16 MiB/s, done.
Resolving deltas: 100% (2923/2923), done.
Updating files: 100% (423/423), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 31.0 MB/s eta 0:00:00


In [4]:
%cd /kaggle/working/RECsys_Challenge2024
! python run_compile_all_cython.py

/kaggle/working/RECsys_Challenge2024
run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/usr/bin/python3'
Compiling [1/11]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1252:
/usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wcpp-Wcpp]8;;]
   17 | #warning "Using deprecated NumPy API, disabl

In [5]:
from Utils.notebookFunctions import *
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit


K_PATH = '/kaggle/working/RECsys_Challenge2024'
GH_PATH = 'TrainedModels/WithoutKFCV'
D_PATH = 'Hybrid/LinearCombination'

np.random.seed(42)

## Import the repository

In [6]:
repo = get_repo_from_github(token)

Repository 'RECsys_Challenge2024' found.


In [7]:
config = {
    'model': 'SecondLinearCombination_Recall',
    'metric': 'Recall',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_SecondLinearCombination_Recall.db',
    'copy_prev_best_params': False,
    'tune_best_params': True,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [8]:
try:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{D_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [9]:
URM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<35736x38121 sparse matrix of type '<class 'numpy.float64'>'
	with 1764607 stored elements in Compressed Sparse Row format>

In [10]:
ICM_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<38121x94331 sparse matrix of type '<class 'numpy.float64'>'
	with 2940040 stored elements in Compressed Sparse Row format>

# Training

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[50])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


## Import the best recommeders previously trained.

In [12]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

recommenders = {
    "ItemKNNCF": ItemKNNCFRecommender,
    "SLIMElasticNet": SLIMElasticNetRecommender,
    "RP3beta": RP3betaRecommender
}

paths_to_best_params = {
    "ItemKNNCF": "KNN",
    "SLIMElasticNet": "SLIM",
    "RP3beta": "GraphBased"
}


loaded_recommenders = {}

for recommender_name, recommender_class in recommenders.items():
    
    start_time = time.time()
    
    print(f"{recommender_name} Model - TRAINING with its best parameters.")
    try:
        recommender = recommender_class(URM_train)
    except Exception as e:
        recommender = recommender_class(URM_train, ICM_all)

    # Extract best parameter values of the relative recommender model.
    with open(f'{GH_PATH}/{paths_to_best_params[recommender_name]}/{recommender_name}Recommender/Optimizing{config["metric"]}/best_params_{recommender_name}_{config["metric"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)
    
    recommender.fit(**best_params)
    
    loaded_recommenders[recommender_name] = recommender
    
    new_time_value, new_time_unit = seconds_to_biggest_unit(time.time() - start_time)
    print("Training done in {:2f}{}.\n".format(new_time_value, new_time_unit))

ItemKNNCF Model - TRAINING with its best parameters.
Similarity column 38121 (100.0%), 2765.91 column/sec. Elapsed time 13.78 sec
Training done in 14.175634sec.

SLIMElasticNet Model - TRAINING with its best parameters.
SLIMElasticNetRecommender: Processed 3815 (10.0%) in 5.00 min. Items per second: 12.71
SLIMElasticNetRecommender: Processed 7788 (20.4%) in 10.00 min. Items per second: 12.98
SLIMElasticNetRecommender: Processed 11715 (30.7%) in 15.00 min. Items per second: 13.01
SLIMElasticNetRecommender: Processed 15659 (41.1%) in 20.00 min. Items per second: 13.05
SLIMElasticNetRecommender: Processed 19621 (51.5%) in 25.00 min. Items per second: 13.08
SLIMElasticNetRecommender: Processed 23674 (62.1%) in 30.00 min. Items per second: 13.15
SLIMElasticNetRecommender: Processed 27611 (72.4%) in 35.01 min. Items per second: 13.15
SLIMElasticNetRecommender: Processed 31017 (81.4%) in 40.01 min. Items per second: 12.92
SLIMElasticNetRecommender: Processed 34609 (90.8%) in 45.01 min. Items 

In [13]:
from Recommenders.Hybrid.LinearCombinationRecommender import LinearCombinationRecommender 

def objective_function_(optuna_trial):
    
    recommender_instance = LinearCombinationRecommender(URM_train, loaded_recommenders.values())
    
    norm = optuna_trial.suggest_categorical('norm', [1, 2, np.inf, -np.inf])
    itemknn_weight = optuna_trial.suggest_float('itemknn_weight', 0.0, 1.0)
    slim_weight = optuna_trial.suggest_float('slim_weight', 0.0, 1.0)
    rp3beta_weight = optuna_trial.suggest_float('rp3beta_weight', 0.0, 1.0)

    # Ensure the weights sum to 1 (Normalization)
    total_weight = itemknn_weight + slim_weight + rp3beta_weight
    itemknn_weight /= total_weight
    slim_weight /= total_weight
    rp3beta_weight /= total_weight
    
    full_hyperp = {
                    "weights": [itemknn_weight, slim_weight,rp3beta_weight],
                    "norm": norm
                  }        
    
    recommender_instance.fit(**full_hyperp)
    
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[50]["RECALL"]

In [14]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_, n_trials=50)

[I 2024-12-30 19:56:46,527] A new study created in RDB with name: hyperparameters_tuning_SecondLinearCombination_Recall_Recall


LinearCombinationRecommender: Fit completed in 0.00 seconds.
EvaluatorHoldout: Processed 35595 (100.0%) in 1.79 min. Users per second: 332


[I 2024-12-30 19:58:33,936] Trial 0 finished with value: 0.2786669523180989 and parameters: {'norm': inf, 'itemknn_weight': 0.8017727686768834, 'slim_weight': 0.06805974943560078, 'rp3beta_weight': 0.9307542841476057}. Best is trial 0 with value: 0.2786669523180989.


LinearCombinationRecommender: Fit completed in 0.00 seconds.
EvaluatorHoldout: Processed 35595 (100.0%) in 1.79 min. Users per second: 332


[I 2024-12-30 20:00:21,366] Trial 1 finished with value: 0.2798327233738146 and parameters: {'norm': 2, 'itemknn_weight': 0.8083539714554404, 'slim_weight': 0.3302456909514888, 'rp3beta_weight': 0.7235596186873365}. Best is trial 1 with value: 0.2798327233738146.


...

[I 2024-12-30 21:20:13,184] Trial 47 finished with value: 0.2857649385222292 and parameters: {'norm': 2, 'itemknn_weight': 0.12438425460031531, 'slim_weight': 0.48023001805246496, 'rp3beta_weight': 0.7301332682139754}. Best is trial 35 with value: 0.28721344892443607.


LinearCombinationRecommender: Fit completed in 0.00 seconds.
EvaluatorHoldout: Processed 35595 (100.0%) in 1.77 min. Users per second: 335


[I 2024-12-30 21:21:59,671] Trial 48 finished with value: 0.2848871590727443 and parameters: {'norm': 2, 'itemknn_weight': 0.4480307272556454, 'slim_weight': 0.7028281045298339, 'rp3beta_weight': 0.9081073750481905}. Best is trial 35 with value: 0.28721344892443607.


LinearCombinationRecommender: Fit completed in 0.00 seconds.
EvaluatorHoldout: Processed 35595 (100.0%) in 1.75 min. Users per second: 338


[I 2024-12-30 21:23:45,038] Trial 49 finished with value: 0.28488609606180826 and parameters: {'norm': 2, 'itemknn_weight': 0.5835796962184489, 'slim_weight': 0.9439109357216166, 'rp3beta_weight': 0.8232349277498803}. Best is trial 35 with value: 0.28721344892443607.


## Some optuna visualizations on recommender parameters

In [15]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [16]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [17]:
if config['tune_best_params']:
    
    loaded_recommenders = {}
    
    for recommender_name, recommender_class in recommenders.items():
        
        start_time = time.time()
        
        print(f"{recommender_name} Model - TRAINING with its best parameters.")
        try:
            recommender = recommender_class(URM_train + URM_validation)
        except Exception as e:
            recommender = recommender_class(URM_train + URM_validation, ICM_all)
    
        # Extract best parameter values of the relative recommender model.
        with open(f'{GH_PATH}/{paths_to_best_params[recommender_name]}/{recommender_name}Recommender/Optimizing{config["metric"]}/best_params_{recommender_name}_{config["metric"]}.json', 'r') as best_params_json:
                best_params = json.load(best_params_json)
        
        recommender.fit(**best_params)
        
        loaded_recommenders[recommender_name] = recommender
        
        new_time_value, new_time_unit = seconds_to_biggest_unit(time.time() - start_time)
        print("Training done in {:2f}{}.\n".format(new_time_value, new_time_unit))

ItemKNNCF Model - TRAINING with its best parameters.
Similarity column 38121 (100.0%), 2524.89 column/sec. Elapsed time 15.10 sec
Training done in 15.574344sec.

SLIMElasticNet Model - TRAINING with its best parameters.
SLIMElasticNetRecommender: Processed 3117 ( 8.2%) in 5.00 min. Items per second: 10.39
SLIMElasticNetRecommender: Processed 6222 (16.3%) in 10.00 min. Items per second: 10.37
SLIMElasticNetRecommender: Processed 9227 (24.2%) in 15.00 min. Items per second: 10.25
SLIMElasticNetRecommender: Processed 12255 (32.1%) in 20.00 min. Items per second: 10.21
SLIMElasticNetRecommender: Processed 15463 (40.6%) in 25.00 min. Items per second: 10.31
SLIMElasticNetRecommender: Processed 18571 (48.7%) in 30.00 min. Items per second: 10.32
SLIMElasticNetRecommender: Processed 22044 (57.8%) in 35.01 min. Items per second: 10.49
SLIMElasticNetRecommender: Processed 24977 (65.5%) in 40.01 min. Items per second: 10.40
SLIMElasticNetRecommender: Processed 27760 (72.8%) in 45.01 min. Items p

In [18]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'{K_PATH}/{GH_PATH}/{D_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)
            
    fit_params = {
    "weights": list(best_params.values())[1:], 
    "norm": best_params["norm"]
    }
    
    recommender_instance = LinearCombinationRecommender(URM_train + URM_validation, loaded_recommenders.values())
    recommender_instance.fit(**fit_params)

LinearCombinationRecommender: Fit completed in 0.00 seconds.


# Testing

Create the recommendations for the submission. 

In [19]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/input/recommender-system-2024-challenge-polimi/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance, f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv')

Submission file saved as /kaggle/working/submission_SecondLinearCombination_Recall_Recall.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [20]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{GH_PATH}/{D_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{config["model"]}_{config["metric"]} tuning results (from kaggle notebook)',
            repo
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{D_PATH}/{config["model"]}Recommender/'\
        f'best_params_{config["model"]}_{config["metric"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json'
    )

File 'TrainedModels/WithoutKFCV/Hybrid/LinearCombination/SecondLinearCombination_RecallRecommender/OptimizingRecall/best_params_SecondLinearCombination_Recall_Recall.json' created successfully.


Save the history of the tuned model.

In [21]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'{GH_PATH}/{D_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db',
        f'Tuning {config["model"]}_{config["metric"]} db updated results (from kaggle notebook)',
        repo
    )

File 'TrainedModels/WithoutKFCV/Hybrid/LinearCombination/SecondLinearCombination_RecallRecommender/OptimizingRecall/history_SecondLinearCombination_Recall_Recall.db' created successfully.


Save the best trained model and its submission.

In [22]:
if config['save_github'] and config['tune_best_params']: 
    upload_file(
                f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv', 
                f'{GH_PATH}/{D_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/Submission/submission_{config["model"]}_{config["metric"]}.csv', 
                f'New {config["model"]}_{config["metric"]} submission (from kaggle notebook)',
                repo
            )

File 'TrainedModels/WithoutKFCV/Hybrid/LinearCombination/SecondLinearCombination_RecallRecommender/OptimizingRecall/Submission/submission_SecondLinearCombination_Recall_Recall.csv' created successfully.
